In [35]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
from photutils.aperture import aperture_photometry, CircularAperture

from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection
import ccdproc as ccdp

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
from convenience_functions import show_image
from astropy.timeseries import BinnedTimeSeries, TimeSeries
from astropy.table import Column

from astropy import units as u
from astropy.coordinates import SkyCoord
from photutils.aperture import SkyCircularAperture


# suppress warnings is not recommended :I
import warnings
warnings.filterwarnings("ignore")

In [38]:
cc_com_reduced = './data/cc_com/reduced/'
tz_boo_reduced = './data/tz_boo/reduced/'

def get_aperture_fluxes(images_dir, image_dim=(3056, 3056), r=50.0):
    images = ImageFileCollection(images_dir)
    images = images.files_filtered(include_path=True)
    
    mask = np.ones(image_dim, dtype=bool)
    mask[1400:1600, 1400:1600] = False
    
    APT_fluxes = []
    for i, path in np.ndenumerate(images):
        # print(path)
        image = CCDData.read(path)

        position = np.transpose((
            image.header['X-CENTROID'],
            image.header['Y-CENTROID']
        ))
        aperture = CircularAperture(position, r=r)
        phot = aperture_photometry(image, aperture)[0]['aperture_sum']
        image.header['APT-FLUX'] = phot.value
        
        APT_fluxes.append(phot.value)
        
    return APT_fluxes

In [39]:
tzboo_fluxes = get_aperture_fluxes(tz_boo_reduced)
tzboo_fluxes

[696743.7145410595, 672665.0032227268, 657920.2438691185, 602233.4861799859]

In [48]:
cccom_fluxes = get_aperture_fluxes(cc_com_reduced)
# cccom_fluxes.write('./results/cccom_DAOresults.csv')
cccom_fluxes

[450925.6694071289, 446639.445057526, 455304.25499404693]

In [45]:
tzboo_DAO = BinnedTimeSeries.read(
    './results/tzboo_DAOresults.csv',
    time_bin_start_column='time_bin_start',
    time_bin_size_column='time_bin_size',
    time_bin_size_unit=u.s
)
apflux_col = Column(name='AP flux', data=tzboo_fluxes)
tzboo_DAO.add_column(apflux_col)
tzboo_DAO.write('./results/tzboo_results.csv')

In [49]:
cccom_DAO = BinnedTimeSeries.read(
    './results/cccom_DAOresults.csv',
    time_bin_start_column='time_bin_start',
    time_bin_size_column='time_bin_size',
    time_bin_size_unit=u.s
)
apflux_col = Column(name='AP flux', data=cccom_fluxes)
cccom_DAO.add_column(apflux_col)
cccom_DAO.write('./results/cccom_results.csv')

In [50]:
cccom_DAO

time_bin_start,time_bin_size,xcentroid,ycentroid,DAO flux,AP flux
,s,,,,
Time,float64,float64,float64,float64,float64
2023-05-05T03:47:49.549,15.0,1553.040442438791,1437.5034251892966,3.319052059965849,450925.6694071289
2023-05-05T04:15:01.570,15.0,1551.4548739944155,1438.76161766664,4.407717909508814,446639.445057526
2023-05-05T04:58:12.580,15.0,1550.1356753563516,1438.672920101272,5.94613799804669,455304.25499404693
